In [1]:
import lxml.etree as ET
import glob
import sys
from MyCapytain.resolvers.cts.local import CtsCapitainsLocalResolver


CORPORA_PATH = "../../data/raw/corpora/**/*"
repositories = list(glob.glob(CORPORA_PATH, recursive=False))
resolver = CtsCapitainsLocalResolver(repositories)

../../data/raw/corpora/lascivaroma_additional-texts/lascivaroma_additional-texts/data/phi1351/phi005/phi1351.phi005.perseus-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng4.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi001/phi0448.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi002/phi0448.phi002.perseus-eng2.xml is 

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0690/phi002/phi0690.phi002.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0690/phi003/phi0690.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1017/phi011/phi1017.phi011.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0660/phi003/phi0660.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1002/phi001/phi1002.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0914/phi001/phi0914.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0917/phi001/phi0917.phi

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1056/phi001/phi1056.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi004/phi1351.phi004.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi002/phi1351.phi002.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0550/phi001/phi0550.phi001.perseus-eng1.xml is not present


In [2]:
data = resolver.getTextualNode("urn:cts:latinLit:phi0660.phi001.perseus-lat2")

In [8]:
import lxml.etree as ET

def get_xslt(citation_scheme):
    additional_xsl = ""
    for level in range(len(citation_scheme)):
        xpath = f"{citation_scheme[level].scope}{citation_scheme[level].xpath}".replace("='?'", "").replace("=\"?\"", "")
        name = citation_scheme[level].name or "unknown"
        if level == 0:
            additional_xsl += "\n" + f"""
        <xsl:template match="{xpath}">
            [REF:{name}.<xsl:value-of select="@n"/>]
            <xsl:apply-templates>
                <xsl:with-param name="previous"><xsl:value-of select="@n"/></xsl:with-param>
            </xsl:apply-templates>
        </xsl:template>"""
        elif level == len(citation_scheme) - 1:
            additional_xsl += "\n" + f"""
        <xsl:template match="{xpath}">
            <xsl:param name="previous" />
            [REF:{name}.<xsl:value-of select="$previous" />.<xsl:value-of select="@n"/>]
            <xsl:apply-templates />
        </xsl:template>"""
            # LAST LEVEL
        else:
            additional_xsl += "\n" + f"""
        <xsl:template match="{xpath}">
            <xsl:param name="previous" />
            [REF:{name}.<xsl:value-of select="$previous" />.<xsl:value-of select="@n"/>]
            <xsl:apply-templates>
                <xsl:with-param name="previous"><xsl:value-of select="$previous" />.<xsl:value-of select="@n"/></xsl:with-param>
            </xsl:apply-templates>
        </xsl:template>"""
    text = """<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform"
    xmlns:xs="http://www.w3.org/2001/XMLSchema"
    exclude-result-prefixes="xs" xmlns:tei="http://www.tei-c.org/ns/1.0"
    version="1.0">
    <xsl:output method="text" encoding="UTF-8"/>
    <xsl:template match="tei:note">
        <xsl:text> </xsl:text>
    </xsl:template>
    <xsl:template match="tei:p|tei:div|tei:seg|tei:l|tei:w|tei:ab">
        <xsl:param name="previous" />
        <xsl:text> </xsl:text><xsl:apply-templates ><xsl:with-param name="previous" select="$previous" /></xsl:apply-templates>
    </xsl:template>
    <xsl:template match="tei:listPerson|tei:teiHeader|tei:label|tei:ref|tei:milestone|tei:orig|tei:abbr|tei:head|tei:title|tei:teiHeader|tei:del|tei:g|tei:bibl|tei:front|tei:back|tei:foreign" />
    {previous}
    <xsl:template match="*[@xml:lang='eng']"/>
    <xsl:template match="*[@type='edition' and @xml:lang='eng']"><xsl:apply-templates /></xsl:template>
</xsl:stylesheet>""".format(previous=additional_xsl)
    return ET.XSLT(ET.fromstring(text)), text

In [9]:
xsl_transform, xsl = get_xslt(data.citation)
print(xsl)
print(xsl_transform(data.xml))

<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform"
    xmlns:xs="http://www.w3.org/2001/XMLSchema"
    exclude-result-prefixes="xs" xmlns:tei="http://www.tei-c.org/ns/1.0"
    version="1.0">
    <xsl:output method="text" encoding="UTF-8"/>
    <xsl:template match="tei:note">
        <xsl:text> </xsl:text>
    </xsl:template>
    <xsl:template match="tei:p|tei:div|tei:seg|tei:l|tei:w|tei:ab">
        <xsl:param name="previous" />
        <xsl:text> </xsl:text><xsl:apply-templates ><xsl:with-param name="previous" select="$previous" /></xsl:apply-templates>
    </xsl:template>
    <xsl:template match="tei:listPerson|tei:teiHeader|tei:label|tei:ref|tei:milestone|tei:orig|tei:abbr|tei:head|tei:title|tei:teiHeader|tei:del|tei:g|tei:bibl|tei:front|tei:back|tei:foreign" />
    

        <xsl:template match="/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n]">
            [REF:book.<xsl:value-of select="@n"/>]
            <xsl:apply-templates>
                <xsl:with-param name="pre